In [1]:
from datasets import load_dataset
# 저장된 train 데이터를 불러옵니다.
dataset = load_dataset("json", data_files="train_dataset.json", split="train")

In [2]:
import trl
import torch
import datasets
import transformers

import pandas as pd
from random import randint
from datasets import Dataset, load_dataset

from trl import SFTTrainer, setup_chat_format
from peft import LoraConfig, PeftConfig, AutoPeftModelForCausalLM

import wandb
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline)

from huggingface_hub import login

import os
import json
from openai import OpenAI

In [3]:
# 학습한 모델을 경로를 지정합니다.
peft_model_id = "./code-llama3-8B-text-to-sql"

# PEFT 어댑터를 통해 사전 학습된 모델을 로드합니다.
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
).to("cuda")

# 토크나이저 로드합니다.
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
tokenizer.padding_side = 'right'  

# 생성을 조금 더 효율적으로 하기 위해 파이프라인을 불러옵니다.
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM'

In [4]:
import warnings
warnings.filterwarnings("ignore")

# 테스트 데이터를 불러옵니다.
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

# 샘플 데이터 설정합니다.
prompt = pipe.tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=False, temperature=0.1, top_k=50, top_p=0.1, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)

print(f"Query:\n{eval_dataset[rand_idx]['messages'][1]['content']}")
print(f"Original Answer:\n{eval_dataset[rand_idx]['messages'][2]['content']}".replace("<|im_end|>", ""))
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")
eval_dataset[rand_idx]['messages'][2]['content'].replace("<|im_end|>", "") == outputs[0]['generated_text'][len(prompt):].strip()

Query:
Task: 지난 학기 STATS 520의 GSI는 누구였나요?
SQL table: CREATE TABLE jobs (
    job_id int,
    job_title varchar,
    description varchar,
    requirement varchar,
    city varchar,
    state varchar,
    country varchar,
    zip int
)

CREATE TABLE ta (
    campus_job_id int,
    student_id int,
    location varchar
)

CREATE TABLE program_requirement (
    program_id int,
    category varchar,
    min_credit int,
    additional_req varchar
)

CREATE TABLE program_course (
    program_id int,
    course_id int,
    workload int,
    category varchar
)

CREATE TABLE requirement (
    requirement_id int,
    requirement varchar,
    college varchar
)

CREATE TABLE gsi (
    course_offering_id int,
    student_id int
)

CREATE TABLE course (
    course_id int,
    name varchar,
    department varchar,
    number varchar,
    credits varchar,
    advisory_requirement varchar,
    enforced_requirement varchar,
    description varchar,
    num_semesters int,
    num_enrolled int,
    has_disc

False

In [10]:
def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt):].strip()
    return (sample["messages"][1]["content"], predicted_answer, sample["messages"][2]["content"])

success_rate = []
for i in tqdm(range(8740, len(eval_dataset))):
    success_rate.append(evaluate(eval_dataset[i]))

with open("./success_rate3.txt", "w") as f:
    for s in success_rate:
        f.write(str(s) + "\n")

  0%|          | 0/4371 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
